验证PIL,opecv,tensor之间的相互转换

In [47]:
img_path = "/home/lyma/FocusOnDepth/datasets/inria/images/0.jpg"
from PIL import Image
import cv2
import torch
from torchvision import transforms
import numpy as np


PIL 相关属性

In [39]:
pil_img = Image.open(img_path)
print(dir(pil_img))
print("image mode:", pil_img.mode)
print("image size:", pil_img.size, "width:", pil_img.width, "height:", pil_img.height)
r, g, b = pil_img.getpixel((1, 1))
print("R:{}, G:{}, B:{} in 1,1".format(r,g,b))

['_Image__transformer', '__array_interface__', '__class__', '__copy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_close_exclusive_fp_after_loading', '_copy', '_crop', '_dump', '_ensure_mutable', '_exclusive_fp', '_exif', '_expand', '_get_safe_box', '_getexif', '_getmp', '_min_frame', '_new', '_open', '_repr_png_', '_seek_check', '_size', 'alpha_composite', 'app', 'applist', 'bits', 'category', 'close', 'convert', 'copy', 'crop', 'custom_mimetype', 'decoderconfig', 'decodermaxblock', 'draft', 'effect_spread', 'entropy', 'filename', 'filter', 'format', 'format_description', 'fp', 'frombytes', 'fromstring', 'get_format_mimetype', 'getbands', 'getb

opencv 相关属性

In [42]:
cv_img = cv2.imread(img_path)
print(dir(cv_img))
print("HWC(row, col, channel):", cv_img.shape, cv_img.size)
b,g,r = cv_img[1,1]
print("R:{}, G:{}, B:{} in 1,1".format(r,g,b))

['T', '__abs__', '__add__', '__and__', '__array__', '__array_finalize__', '__array_function__', '__array_interface__', '__array_prepare__', '__array_priority__', '__array_struct__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__complex__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__ilshift__', '__imatmul__', '__imod__', '__imul__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__irshift__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__lshift__', '__lt__', '__matmul__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift_

用torch读、转换PIL,opencv

In [40]:
transform_totensor = transforms.Compose([
    transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
    ])
pil_tensor = transform_totensor(pil_img)
print("CHW:", pil_tensor.shape, pil_tensor.min(), pil_tensor.max())
cv_tensor = transform_totensor(cv_img)
print("CHW:", cv_tensor.shape, cv_tensor.min(), cv_tensor.max())

CHW: torch.Size([3, 540, 960]) tensor(0.) tensor(1.)
CHW: torch.Size([3, 540, 960]) tensor(0.) tensor(1.)


PIL-->Tensor直接用ToPILImage()实现
.convert('RGB')不是必要的！！！

In [46]:
pil_img_transformed = transforms.ToPILImage()(pil_tensor)#.convert('RGB')
pil_img_transformed.save("pil_img_transformed.jpg")

opencv-->Tensor
np.transpose是必要的！！！

In [51]:
cv_img_transformed = cv_tensor.numpy()*255
cv_img_transformed = cv_img_transformed.astype('uint8')
cv_img_transformed = np.transpose(cv_img_transformed, (1,2,0))
cv2.imwrite("cv_img_transformed.jpg",cv_img_transformed)

True

PIL-->opencv

In [52]:
pil_to_cv_img = cv2.cvtColor(np.asarray(cv_img),cv2.COLOR_RGB2BGR)  
b,g,r = pil_to_cv_img[1,1]
print("R:{}, G:{}, B:{} in 1,1".format(r,g,b))

R:0, G:1, B:1 in 1,1


opencv-->PIL

In [53]:
cv_to_pil_img = Image.fromarray(cv_img)
r, g, b = cv_to_pil_img.getpixel((1, 1))
print("R:{}, G:{}, B:{} in 1,1".format(r,g,b))

R:0, G:1, B:1 in 1,1
